In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate flash_attn
!pip install qwen_vl_utils av

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2g84ht49
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-2g84ht49
  Resolved https://github.com/huggingface/transformers to commit 21fac7abba2a37fae86106f87fcf9974fd1e3830
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9677029 sha256=ad326991939ac1b0e17c8e9cb1978c72380d3067c70fcc9b1505aa12d72f70cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-iwlu7dhh/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
  Created wheel for flash_attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

In [3]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

In [4]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2", device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

In [5]:
processor = AutoProcessor.from_pretrained(model_name)

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [10]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/content/video.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "Why did the doctor mark on the top of knee area?."},
        ],
    }
]

In [11]:
messages

[{'role': 'user',
  'content': [{'type': 'video',
    'video': '/content/video.mp4',
    'max_pixels': 151200,
    'fps': 1.0},
   {'type': 'text',
    'text': 'Why did the doctor mark on the top of knee area?.'}]}]

In [12]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [13]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

In [14]:
inputs = inputs.to("cuda")

In [15]:
generated_ids = model.generate(**inputs, max_new_tokens=512)

In [16]:
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

In [17]:
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [18]:
print(output_text)

["In the video, the doctor is marking a specific area on the patient's knee. This is typically done to ensure that the incision line is accurately placed and that the surgeon can accurately access the knee joint during the surgery. The marking helps to guide the surgeon in making precise cuts and movements during the procedure."]
